### Программа для получения информации по всем строящимся жк в РК

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import urlparse, urljoin
import pandas as pd
import unicodedata

In [24]:
def parse_first_page_small(city, amount): ### Функция парсит все ЖК из города, где меньше 13 строящихся ЖК. Передаётся явное количество ЖК, чтобы забрать именно столько
                                          ###  Это нужно из-за отсутствия Пейджера в нижней части страницы. В остальных случаях мы ориентируемся по нему.
    url = f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1'
    parsed_url = urlparse(url)
    domain = parsed_url.scheme + '://' + parsed_url.netloc
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        # join domain to path
        full_url = urljoin(domain, link['href']) 
        links.append(full_url)
    page_links = [] ### Здесь будем забирать после слова "новостройки" ровно amount жк
    page_links = links[links.index('https://special.kolesa.group/novostroiki') + 1 : links.index('https://special.kolesa.group/novostroiki') + amount + 1]
    return page_links

In [39]:
def parse_first_page(city): ### Мы будем парсить первую и вторую страницу города отдельно из-за их отличия в html коде от других страниц
    url = f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1'
    parsed_url = urlparse(url)
    domain = parsed_url.scheme + '://' + parsed_url.netloc
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        # join domain to path
        full_url = urljoin(domain, link['href']) 
        links.append(full_url)
    page_links = []  ### Будем забирать от "новостройки" до пейджера.
    page_links = links[links.index('https://special.kolesa.group/novostroiki') + 1 : links.index(f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1')]
    return page_links

In [5]:
def parse_second_page(city):
    url = f"https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1&page=2"
    parsed_url = urlparse(url)
    domain = parsed_url.scheme + '://' + parsed_url.netloc
    response = requests.get(url) 
    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a', href=True):
        # join domain to path
        full_url = urljoin(domain, link['href']) 
        links.append(full_url)
    page_links = []
    page_links = links[links.index('https://special.kolesa.group/novostroiki') + 1 : links.index(f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1')]
    return page_links

In [26]:
def pages_count(city): ### Функция забирает из Пейджера количество страниц с ЖК в данном городе
    url = f"https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    pager = soup.select('.complex-pager')
    pure_pager = pager[0].text.replace('  ', '').replace('\n', '') ### Чистим html мусор
    if len(pure_pager) > 13: ### Если количество страниц больше 8, то из символов, к примеру, '1' и '4' мы делаем число 14
        pages = int(pure_pager[pure_pager.index('Д') - 1]) + int(pure_pager[pure_pager.index('Д') - 2]) * 10
    else: ### иначе, просто забираем последнюю цифру перед словом "Дальше" и переводим её в int
        pages = int(pure_pager[pure_pager.index('Д') - 1])
    return pages

In [27]:
def parse_other_pages(city, list_of_pages): ### Функция парсит все остальные страницы по данному городу(начиная с 3)
    all_pages = []
    pages_amount = pages_count(city)
    for page in range(3, pages_amount + 1): ### Цикл для прохода по всем остальным страницам новостроек
        url = f"https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1&page={page}"
        parsed_url = urlparse(url)
        domain = parsed_url.scheme + '://' + parsed_url.netloc
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = []
        for link in soup.find_all('a', href=True):
         #join domain to path
            full_url = urljoin(domain, link['href']) 
            links.append(full_url)
        page_links = []
        page_links = links[links.index('https://special.kolesa.group/novostroiki') + 1 : links.index(f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1') - 1]
        list_of_pages.append(page_links)

In [28]:
def check_buildings_amount(city): ### Функция считает количество ЖК в данном городе
    url = f'https://krisha.kz/complex/search/{city}/?ceiling=&state[]=1'
    response1 = requests.get(url)
    soup = BeautifulSoup(response1.text, 'html.parser')
    res = soup.select(".complex-heading__count")
    x = res[0].text.replace('\n', '').replace('  ', '') ### Чистим мусор
    return int(x[0 : x.index('н')]) ### Забираем из строки вида "166новостроек" именно "166"

In [29]:
def parse_building_page(url): ### Основная функция, которая возвращает список со всей информацией по ЖК
    building_info = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    name = get_building_name(soup)
    building_info.append(name)
    price = get_price_offer(soup)
    building_info.append(price)
    main_info = get_building_info(soup)
    building_info.append(main_info)
    parameters = get_building_parameters(soup)
    building_info.append(parameters)
    building_info.append(url)
    return building_info

In [30]:
def get_building_parameters(soup): ### Функция возвращает Словарь с параметрами о ЖК (Класс жилья, этажность и т.д.)
    parameters = {}
    info_title = soup.find_all('dt') ### Title(пример: "Класс:")
    info_data = soup.find_all('dd')  ### Data(пример: "Комфорт")
    iter = 0
    for elem in info_title:
        if (elem.text).find('Внутри жилого комплекса') >= 0: ### Не забираем лишней информации.
            break
        else:
            parameters[elem.text] = info_data[iter].text
            iter += 1
    return parameters

In [11]:
def get_building_name(soup): ### Функция забирает со страницы название недвижимости. Возвращаемое значение - str
    name_dirty = soup.select("h1")
    return ((name_dirty[0].text).replace('  ', '')).replace('\n', '')

In [12]:
def to_float(string_price): ### Переводим цену в тип float. Если цена в виде "1.4млн", то переводим её в "1400000"
    if string_price.find('млн') >= 0:
        return float(string_price[0:string_price.index('м')]) * 1000000
    else:
        return float(string_price)

In [31]:
def get_price_offer(soup): ### Функция забирает со страницы стоимость недвижимости. возвращемое значение - str
    price_dirty = soup.select(".offer__price")
    rexp = r'(?<=от\s).*(?=<\w)'
    price_clean_ungrouped = (re.search(rexp, str(price_dirty)))
    if price_clean_ungrouped:
        price_clean = price_clean_ungrouped.group()
    else:
        return 'Undefined' ### если цена не указана
    return to_float(price_clean.replace('\xa0', '')) ### Избавляемся от дополнительных html-символов и возвращаем float

In [14]:
def get_building_info(soup): ### Функция забирает со страницы информацию о недвижимости. Возвращает list of strs
    info_dirty = soup.select(".complex__sidebar-info-text")
    info_clean = []
    for i in range(4):
        text = info_dirty[i].text
        info_clean.append((text.replace('  ', '')).replace('\n', ''))
    return info_clean

In [32]:
def parse_city_links(city): ### Функция собирает ссылки на все жк города
    list_of_pages = []
    buildings_amount = check_buildings_amount(city)
    if buildings_amount < 13: ### если у населенного пункта только 1 страница
        first_page_links = parse_first_page_small(city, buildings_amount)
        list_of_pages.append(first_page_links)
        return list_of_pages
        
    
### Достаём ссылки первой страницы отдельно вне цикла из-за отличия её от других страниц
    first_page_links = parse_first_page(city)
    
### Заносим ссылки первой страницы в список, где будут храниться все ссылки на все новостройки
    list_of_pages.append(first_page_links)
    
### Достаём ссылки на ЖК со второй страницы отдельно вне цикла из-за отличия её от других страниц
    second_page_links = parse_second_page(city)
    
### Заносим ссылки второй страницы в список, где будут храниться все ссылки на все новостройки    
    list_of_pages.append(second_page_links)
    
### Достаём ссылки на ЖК со всех остальных страниц и сразу заносим в общий список
    other_pages_links = parse_other_pages(city, list_of_pages)
    return list_of_pages

In [33]:
def parse_city_information(city): ### Функция получает всю информацию по всем строящимся ЖК с данного города
    links = parse_city_links(city)
    dict_links = {}
    dict_links = dict(enumerate(links, start = 1))
    information_list = []
    for key in dict_links.keys():
        for elem in dict_links[key]:
            information_list.append(parse_building_page(elem))
    return information_list

In [34]:
def to_xlsx(city, writer): ### Функция заносит информацию о строящихся ЖК в городе в excel файл в отдельный лист, который соотвествует городу.
    information = parse_city_information(city)
    d1 = {}
    d1["name"], d1["price"], d1["info"], d1["status"], d1["year_of_building"], d1["address"], d1["developer"], d1["class"], d1["parameters"], d1["link"] = [], [], [], [], [], [], [], [], [], []
    for elem in information:
        d1["name"].append(elem[0])
        d1["price"].append(elem[1])
        d1["info"].append(elem[2])
        d1["status"].append(elem[2][0])
        d1["year_of_building"].append(elem[2][1])
        d1["address"].append(elem[2][2])
        d1["developer"].append(elem[2][3])
        d1["class"].append(elem[3]['Класс жилья'])
        #elem[3].pop('Класс жилья', None)
        d1["parameters"].append(elem[3])
        d1["link"].append(elem[4])
    df = pd.DataFrame(d1)
    df.to_excel(writer, sheet_name=f"{city}")
    return df

In [36]:
def get_cities_list(): ### Функция парсит все названия городов в формате сайта Крыша.кз (например: "almaty", "astana")
    url = "https://krisha.kz/complex/search/foreign/?ceiling=&state[]=1"
    response1 = requests.get(url)
    soup = BeautifulSoup(response1.text, 'html.parser')
    res = soup.select(".seo-links__city_list") ### Забираем из списка городов в нижней части страницы
    x = res[0].find_all('a') ### <a href="/complex/search/almaty/"> Алматы </a> Из такой строки мы забираем именно "almaty"
    cities_list = []
    for elem in x:
        cities_list.append((str(elem)[str(elem).find('search') + 7:str(elem).index('>') - 2]))
    return cities_list

In [38]:
#import time
#from random import randint
#from time import sleep

def current_information(name_of_document): ### Функция пробегается по всем городам и заносит данные в указанный Excel файл по разным листам
    cities_list = get_cities_list()
    writer = pd.ExcelWriter(name_of_document, engine = "xlsxwriter")
    for elem in cities_list:
        #value_sleep = randint(1,5) ### Нужно в случае, если Крыша блокирует парсинг
        #sleep(value_sleep)
        print(elem)
        to_xlsx(elem, writer)
    writer.close()
    

In [40]:
current_information('All_regions.xlsx')

almaty
astana
shymkent
karagandinskaja-oblast-abaj
almatinskaja-oblast-abaj
akmol
aksaj
aktau
aktobe
atyrau
besagash-dzerzhinskoe
boroldaj-burundaj
borovoe
zhanaozen
irgeli
karabulak
karaganda
kaskelen
kokshetau
konaev
kostanaj
kosshy
kyzylorda
otegen-batyr
pavlodar
petropavlovsk
semej
talgar
taldykorgan
taraz
tuzdybastau-kalinino
turkestan
uralsk
ust-kamenogorsk
shhuchinsk
ekibastuz
